In [4]:
import csv_to_sqlite
from pathlib import Path
Path('air.db').touch()
import pandas as pd
#asthma = pd.read_csv("Asthma & Air Quality Data.csv")
#asthma.to_sql('final', conn, if_exists='append', index = False)

#d = c.execute('''SELECT * FROM final''').fetchall()

from sqlalchemy import create_engine 
  
# SQLAlchemy connectable 
cnx = create_engine('sqlite:///air.db').connect() 
  
# table named 'contacts' will be returned as a dataframe. 
df = pd.read_sql_table('final', 'sqlite:///air.db')
df['AsthmaERVisits']=df['AsthmaERVisits'].str.replace(',','')
df['AsthmaERVisits'] = df['AsthmaERVisits'].astype(float)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Borough               42 non-null     object 
 1   Geography-UHF42       42 non-null     object 
 2   Latitude              42 non-null     float64
 3   Longitude             42 non-null     float64
 4   AirQualityIndicator   42 non-null     object 
 5   Meanmcgpercubicmeter  42 non-null     int64  
 6   AgeGroup              42 non-null     object 
 7   AsthmaERVisits        42 non-null     float64
dtypes: float64(3), int64(1), object(4)
memory usage: 2.8+ KB


In [24]:
import json
import pandas as pd
import geopandas as gpd
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure, output_file, save
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer


In [25]:
dfr = gpd.read_file('uhf42.geojson')
#df.info()

In [26]:
#Change the data type and column name to match the geojson file 
df['uhf_neigh'] = df['uhf_neigh'].astype(str)
#df.rename(index=str, columns={'Geography-UHF42': 'uhf_neigh'}, inplace=True)
#Turn the GeoJson file into a data frame and merge it with df 
data = 'uhf42.geojson'
dfr = gpd.read_file(data)
merge = dfr.merge(df, how='left', on='uhf_neigh')
#Turn the merged data frame back into a json file  
merged_json = json.loads(merge.to_json())
json_data = json.dumps(merged_json)
print(merge)

    cartodb_id  objectid        borough                             uhf_neigh  \
0            1         1            N/A                                  None   
1            5         5          Bronx                  Pelham - Throgs Neck   
2           18        18       Brooklyn         Coney Island - Sheepshead Bay   
3           32        32         Queens                  Flushing - Clearview   
4           37        37         Queens                               Jamaica   
5           38        38         Queens                      Southeast Queens   
6           43        43  Staten Island             South Beach - Tottenville   
7           39        39         Queens                              Rockaway   
8           24        24      Manhattan                       Upper East Side   
9           27        27      Manhattan              Greenwich Village - Soho   
10          30        30         Queens            Long Island City - Astoria   
11          35        35    

In [27]:
geosource = GeoJSONDataSource(geojson = json_data)
#set the color palette 
palette = brewer['PuRd'][8]
palette = palette[::-1]
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 4000,  nan_color = '#d9d9d9')
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
border_line_color='black',location = (0,0), orientation ='horizontal')
#Set the size and title of the graph
p = figure(title = 'Asthma ER Visits and Air Quality', plot_height = 700 , plot_width = 700, toolbar_location = None, 
          tooltips=[
         ("Neighborhood", "@uhf_neigh"),
         ("Air Quality", "@Meanmcgpercubicmeter"),
         ("Asthma ER Visits","@AsthmaERVisits")])
#Makes it so there are no grid lines
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.patches('xs','ys', source = geosource,fill_color = {'field'     :'AsthmaERVisits', 'transform' : color_mapper},
         line_color = 'black', line_width = 0.25, fill_alpha = 1)
p.add_layout(color_bar, 'below')
output_file("test.html")
save(p)

'C:\\Users\\Diana\\Desktop\\test.html'